# 导入库

In [1]:
import pandas as pd
from sensirion_database_access import DatabaseAccess
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
import sensirion_database_access
import re
from lxml import etree
import  xml.dom.minidom
from xml.dom import minidom
import xml.etree.ElementTree as ET
import warnings
import sensirion_fastedf as sf
import glob
import seaborn as sns
import sensirion_nbconflux
from sensirion_pmcalib_eval_tools.database_analysis_tool import *
# import Ipython
import numpy as np

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.precision = 2

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

# 取数据函数

In [4]:
class Xmlparser:
    def __init__(self,pthroot,MC):
        self.MC = MC
        if isinstance(pthroot,str):
            filepath = []
            for root,dirs,files in os.walk(pthroot):
                for file in files:
                    filepath.append(os.path.join(root,file))
            if [f for f in filepath if all(x in f for x in [MC,"binningcfg"])]:
                self.path = [f for f in filepath if all(x in f for x in [MC,"binningcfg"])]
            else:
                print("No {} files in the path".format(MC))
        elif isinstance(pthroot,dict):
            self.path = pthroot
        else:
            print("Input as str or dict")
            
    def xml_HarSoft_Mapping(self,xmlpth):
        #softbin和hardbin的对应关系
        FFMC03_XML = xmlpth
        tree = ET.parse(FFMC03_XML)
        root = tree.getroot()
        SofHar = pd.DataFrame()
        #SoftBin,HardBin对照
        for child in root.findall("./SoftBins"):
            for bin in child:
                BinCode = bin.attrib.get('BinCode', None)[2:]
                HardBinCode = bin.attrib.get('HardBinCode', None)
                BinDescription = bin.attrib.get('BinDescription',None)
                SoftBin_HardBin = '_'.join(['PM',BinCode,HardBinCode,BinDescription])
                SofHar.loc[BinCode,'HardBin'] = HardBinCode
                SofHar.loc[BinCode,'Description'] = BinDescription
                SofHar.loc[BinCode,'So_Har'] = SoftBin_HardBin
        
        

        #TestId + HardBin + SoftBin + 描述
        TestId_par = pd.DataFrame()
        #MC03 test id 信息
        for child in root.findall(r".//MC03"):
            for bin in child:
                for Id in bin:
                    TestId = Id.tag
                    TestName = Id.attrib['TestName']
                    Unit = Id.attrib['Unit']
                    LBin = Id.attrib['LBin']
                    UBin = Id.attrib['UBin']
                    LTL = Id.attrib['LTL']
                    UTL = Id.attrib['UTL']
                    Comment = Id.attrib['Comment']
                    TestId_par.loc[TestId,'TestName'] = TestName
                    TestId_par.loc[TestId,'Unit'] = Unit
                    TestId_par.loc[TestId,'LBin'] = LBin
                    TestId_par.loc[TestId,'UBin'] = UBin
                    TestId_par.loc[TestId,'LTL'] = LTL
                    TestId_par.loc[TestId,'UTL'] = UTL
                    TestId_par.loc[TestId,'Comment'] = Comment
        self.TestId_par = TestId_par
        TestId_par.sort_values('UBin',inplace=True)

        return SofHar, TestId_par

# BinCode对应的参数(不需要类实例化)

In [5]:
xmlpth = r"C:\SVN\SEN5X_Calibration_ini\3.000.535-004\MC03_CN-100340_3.000.535-004.binningcfg.xml"
softbin = "0x5F"
SofHar, TestId_par = Fairfield.xml_HarSoft_Mapping(xmlpth)
TestId_par = TestId_par.query("UBin == @softbin or LBin == @softbin")
# TestId_par.to_csv("softbin_info.csv")
TestId_par

,TestName,Unit,LBin,UBin,LTL,UTL,Comment
T008044,AlphaNc_4p0,1_per_cm3,0x5F,0x5F,none,none,added by BinningCfgPatcher
T008043,AlphaMc_4p0,ug_per_m3,0x5F,0x5F,none,none,added by BinningCfgPatcher
T008042,AlphaNc_2p5,1_per_cm3,0x5F,0x5F,none,none,added by BinningCfgPatcher
T008041,AlphaMc_2p5,ug_per_m3,0x5F,0x5F,none,none,added by BinningCfgPatcher
T008040,AlphaNc_1p0,1_per_cm3,0x5F,0x5F,none,none,added by BinningCfgPatcher
T008039,AlphaMc_1p0,ug_per_m3,0x5F,0x5F,none,none,added by BinningCfgPatcher
T008038,AlphaNc_0p5,1_per_cm3,0x5F,0x5F,none,none,added by BinningCfgPatcher
T008037,AlphaMc_0p5,ug_per_m3,0x5F,0x5F,none,none,added by BinningCfgPatcher
T008016,Omega,Ticks16Float,0x5F,0x5F,0.745,0.85,was: 0.765
T085005,Pm2p5NumbConcDutMean,1_per_cm3,0x5F,0x5F,none,none,added by BinningCfgPatcher


In [6]:
# SofHar

# 时间段设定:

In [9]:
# 要读取的列
column_list = ["TrayId","DeviceId"]

# 时间段
filter_list = ["StartDateTime BETWEEN '2021-12-14 00:00:00' AND '2022-01-15 23:59:59'"]


# DB:
    # FairfieldDB
    # SEN4XDB
    # SEN5XDB
    # SAPS30_LSHA_DB
    # "JuraDB"

FF = Fairfield(column_list,filter_list,return_testnames=True,include_limits=False,M="MC03",dbname="SAPS30_LSHA_DB")
mc03 = FF.data

In [10]:
mc03.head()

,ProductionOrder,BinCode,SubstrateSessionId,PmTrayId,DeviceId,DeviceIdInternal,TrayId,day,week,month
StartDateTime,,,,,,,,,,
2022-01-10 13:36:13,0002104165,7A,48.0,97,A602A422,2.79e+09,1.0,2022-01-10,2,2022-01
2022-01-10 13:36:13,0002104165,71,48.0,97,None,NaN,1.0,2022-01-10,2,2022-01
2022-01-10 13:36:13,0002104165,71,48.0,97,None,NaN,1.0,2022-01-10,2,2022-01
2022-01-10 13:36:13,0002104165,01,48.0,97,AAAD756C,2.86e+09,1.0,2022-01-10,2,2022-01
2022-01-10 13:36:13,0002104165,71,48.0,97,None,NaN,1.0,2022-01-10,2,2022-01


In [11]:
mc03["PmTrayId"].unique()

array(['97'], dtype=object)

# upload script to Confluence

In [ ]:
html, resources = sensirion_nbconflux.notebook_to_page(r'C:\Sensirion\10_product\60_PM25\110_experiment\20211126_G2A_Omega\G2A_omega.ipynb', "https://confluence/display/GPEC/E-CN-0051_01", username="osun",ssl_verify=False)

# winsound: 声音

In [1]:
import winsound

In [5]:
winsound.Beep(1000,2000)

# 设置报错(警告)等级

In [12]:
import warnings
warnings.filterwarnings("ignore")

# 对比两个DataFrame的区别

In [ ]:
import datacompy

In [ ]:
compare = datacompy.Compare(df1=TestId_par_new, 
                            df2=TestId_par_old,
                            on_index=True,
                            df1_name="SEN4X_new",
                            df2_name="SEN4X_old")
print(compare.report())

# 查看jupyternotebook默认目录

In [1]:
import os

In [2]:
os.path.abspath(".")

'C:\\Sensirion\\10_product\\60_PM25\\110_experiment'